## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configparser import ConfigParser
config = ConfigParser()
config.read('../config.cfg')

# Configure gmaps API key
g_key=config.get('google','api_key')
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Umm lajj,SA,25.0213,37.2685,80.47,60,0,4.45,clear sky
1,1,Dikson,RU,73.5069,80.5464,-6.95,93,36,8.63,scattered clouds
2,2,Castro,BR,-24.7911,-50.0119,74.75,59,56,4.85,broken clouds
3,3,Punta arenas,CL,-53.1500,-70.9167,57.31,33,40,17.27,scattered clouds
4,4,Bambous virieux,MU,-20.3428,57.7575,75.45,73,40,9.22,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_df=city_data_df.loc[(city_data_df["Max Temp"]>=min_temp)&(city_data_df["Max Temp"]<=max_temp)]

In [11]:
# 4a. Determine if there are any empty rows.
preferred_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Umm lajj,SA,80.47,clear sky,25.0213,37.2685,
4,Bambous virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
7,Wewak,PG,81.16,broken clouds,-3.5534,143.6268,
16,Hilo,US,79.29,broken clouds,19.7297,-155.0900,
19,Inhambane,MZ,75.18,few clouds,-23.8650,35.3833,
21,Hithadhoo,MV,82.18,broken clouds,-0.6000,73.0833,
27,Denpasar,ID,78.46,moderate rain,-8.6500,115.2167,
30,Sao filipe,CV,77.43,scattered clouds,14.8961,-24.4956,
35,Bengkulu,ID,75.15,overcast clouds,-3.8004,102.2655,
39,Biak,ID,77.00,overcast clouds,-0.9131,122.8766,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"]=f"{lat},{lng}"
    hotels=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping....")       

Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Umm lajj,SA,80.47,clear sky,25.0213,37.2685,Moon light Furnished Units
4,Bambous virieux,MU,75.45,scattered clouds,-20.3428,57.7575,Casa Tia Villa
7,Wewak,PG,81.16,broken clouds,-3.5534,143.6268,In Wewak Boutique Hotel
16,Hilo,US,79.29,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Inhambane,MZ,75.18,few clouds,-23.8650,35.3833,Oceano


In [16]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \N{DEGREE SIGN}F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30,31),zoom_level=2)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))